In [ ]:
import pandas as pd
train_df = pd.read_csv('../input/scibert-data/train.csv',delimiter=',',header=None,names=['text','label'])

#train.label = train.label.apply({'CL':0,'CR':1,'DC':2,'DS':3,'LO':4, 'NI':5, 'SE':6}.get)
val_df = pd.read_csv('../input/scibert-data/validation.csv',delimiter=',',header=None,names=['text','label'])

#val.label = val.label.apply({'CL':0,'CR':1,'DC':2,'DS':3,'LO':4, 'NI':5, 'SE':6}.get)

#test = pd.read_csv('/content/drive/MyDrive/sdpra2021/test.csv',delimiter=',',
 #                header=None,names=['text'])

In [ ]:
train_df.text[0]

In [ ]:
train_df.head()

In [ ]:
val_df.head()

In [ ]:
import re
import nltk
#nltk.download('stopwords')
#nltk.download('wordnet')
#nltk.download('punkt')
from nltk.corpus import stopwords
#from nltk.stem.porter import PorterStemmer
#from nltk.stem.wordnet import WordNetLemmatizer
def clean(data):
    corpus = []
    rows = data.shape[0]
    for i in range(0, rows):
        title = re.sub('[^a-zA-Z]', ' ', data['text'][i])
        title = title.lower()
        title = title.split()
      #ps = PorterStemmer()
        #lemm=WordNetLemmatizer()  
        all_stopwords = stopwords.words('english')
#         text = ''.join([word for word in text if word in string.printable])
      #title = [ps.stem(word) for word in title if not word in set(all_stopwords)]
        #title = [lemm.lemmatize(word) for word in title if not word in set(all_stopwords)]
        title = ' '.join(title)
        corpus.append(title)
    return corpus

In [ ]:
corpus_train = clean(train_df)
corpus_val = clean(val_df)

In [ ]:
train_df['text'] = corpus_train
val_df['text'] = corpus_val

In [ ]:
train_df.head()

In [ ]:
train_df.text[0]

In [ ]:
val_df.text[0]

In [ ]:
print(len(train_df))
print(len(val_df))

In [ ]:
from sklearn.model_selection import train_test_split
X = val_df.iloc[:, :-1]
y = val_df.iloc[:, -1]

X_val, X_test, y_val, y_test = train_test_split(X, y, test_size = 0.5, random_state = 42)

In [ ]:
X_val.head()

In [ ]:
y_val.head()

In [ ]:
val=pd.concat([X_val, y_val], axis=1)
test=pd.concat([X_test, y_test], axis=1)

In [ ]:
val.head()

In [ ]:
test.head()

In [ ]:
train=train_df

In [ ]:
train.head()

In [ ]:

print('Training set size:',train.shape)
#print('Testing set size:',test.shape)
print('validation set size:',val.shape)

In [ ]:
train.label = train.label.apply({'CL':0,'CR':1,'DC':2,'DS':3,'LO':4, 'NI':5, 'SE':6}.get)
val.label = val.label.apply({'CL':0,'CR':1,'DC':2,'DS':3,'LO':4, 'NI':5, 'SE':6}.get)

In [ ]:
train.head()

In [ ]:

max(train.text.apply(len))

In [ ]:
test.head()

In [ ]:
test.label = test.label.apply({'CL':0,'CR':1,'DC':2,'DS':3,'LO':4, 'NI':5, 'SE':6}.get)

In [ ]:
test.head()

In [ ]:
import pandas as pd
from torch.utils.data import Dataset,DataLoader

class RFDataset(Dataset):
  def __init__(self,text,label,tokenizer,max_len):
    self.text = text
    self.label = label
    self.tokenizer = tokenizer
    self.max_len = max_len
  
  def __len__(self):
    return len(self.text)
  
  def __getitem__(self,item):
    text = str(self.text[item])
    label = self.label[item]

    encoding = self.tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length = self.max_len,
        return_token_type_ids = False,
        padding = 'max_length',
        return_attention_mask= True,
        return_tensors='pt',
        truncation=True
    )

    return {
        'text' : text,
        'input_ids' : encoding['input_ids'].flatten(),
        'attention_mask' : encoding['attention_mask'].flatten(),
        'label' : torch.tensor(label,dtype=torch.long)

    }

In [ ]:
import numpy as np

from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',np.unique(train.label.values),train.label.values)
class_weights

In [ ]:
def create_data_loader(df,tokenizer,max_len,batch_size):
    ds = RFDataset(
      text = df.text.to_numpy(),
      label = df.label.to_numpy(),
      tokenizer = tokenizer,
      max_len = max_len
    )

    return DataLoader(ds,
                    batch_size = batch_size,
                    shuffle = True,
                    pin_memory=True)

In [ ]:
from transformers import *
device = 'cuda'
PRE_TRAINED_MODEL_NAME = 'allenai/scibert_scivocab_uncased'
tokenizer = AutoTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME,return_dict=False)

In [ ]:
BATCH_SIZE = 32
MAX_LEN = 128
train_data_loader = create_data_loader(train,tokenizer,MAX_LEN,BATCH_SIZE)
val_data_loader = create_data_loader(val,tokenizer,MAX_LEN,BATCH_SIZE)
#print(train_data_loader)

In [ ]:
bert_model = AutoModel.from_pretrained(PRE_TRAINED_MODEL_NAME,return_dict=False)

In [ ]:

import torch.nn as nn
class RFClassifier(nn.Module):
  def __init__(self, n_classes,pre_trained):
    super(RFClassifier, self).__init__()
    self.auto = AutoModel.from_pretrained(pre_trained,return_dict=False)
    self.drop = nn.Dropout(p=0.5)
    #self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
    self.out1 = nn.Linear(self.auto.config.hidden_size, 128)
    self.drop1 = nn.Dropout(p=0.5)
    self.relu = nn.ReLU()
    self.out = nn.Linear(128, n_classes)
  
  def forward(self, input_ids, attention_mask):
    _,pooled_output = self.auto(
      input_ids=input_ids,
      attention_mask=attention_mask
    )
    #output = self.relu(pooled_output)
    output = self.drop(pooled_output)
    output = self.out1(output)
    output = self.relu(output)
    output = self.drop1(output)
    return self.out(output)

In [ ]:
model = RFClassifier(7,'allenai/scibert_scivocab_uncased')
model = model.to(device)

In [ ]:
EPOCHS = 5
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)

loss_fn = nn.CrossEntropyLoss().to(device)

In [ ]:
def train_epoch(model,data_loader,loss_fn,optimizer,device,scheduler,n_examples):
    model = model.train()
    losses = []
    correct_predictions = 0

    for data in data_loader:
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        labels = data['label'].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            #labels_ids=labels
            )
        _, preds = torch.max(outputs, dim=1)
        loss = loss_fn(outputs,labels)

        correct_predictions += torch.sum(preds == labels)
        losses.append(loss.item())

        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    return correct_predictions.double() / n_examples, np.mean(losses)

In [ ]:
def eval_model(model, data_loader, loss_fn, device, n_examples):
    model = model.eval()
    losses = []
    correct_predictions = 0
    with torch.no_grad():
        for d in data_loader:
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            labels = d["label"].to(device)
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
        #labels_ids=labels
            )
            _, preds = torch.max(outputs, dim=1)
            loss = loss_fn(outputs, labels)
            correct_predictions += torch.sum(preds == labels)
            losses.append(loss.item())
    return correct_predictions.double() / n_examples, np.mean(losses)

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
from collections import defaultdict
import torch
 
history = defaultdict(list)
best_accuracy = 0
for epoch in range(EPOCHS):
 
 
    start_time = time.time()
    train_acc,train_loss = train_epoch(
        model,
        train_data_loader,
        loss_fn,
        optimizer,
        device,
        scheduler,
        len(train)
    )
   
  
    val_acc,val_loss = eval_model(
        model,
        val_data_loader,
        loss_fn,
        device,
        len(val)
    )
  
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'Train Loss {train_loss} accuracy {train_acc}')
    print(f'Val Loss {val_loss} accuracy {val_acc}')
    print()

    history['train_acc'].append(train_acc)
    history['train_loss'].append(train_loss)
    history['val_acc'].append(val_acc)
    history['val_loss'].append(val_loss)

    if val_acc > best_accuracy:
        best_accuracy = val_acc
torch.save(model.state_dict(),'scibert.bin')

In [ ]:
test.head()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history['train_acc'], label='train accuracy')
plt.plot(history['val_acc'], label='validation accuracy')
plt.title('Training history')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()

In [ ]:
val_acc, _ = eval_model(
  model,
  val_data_loader,
  loss_fn,
  device,
  len(val) #Change it to test when you have the test results
)
val_acc.item()

In [ ]:
test_data_loader = create_data_loader(test,tokenizer,MAX_LEN,BATCH_SIZE)

In [ ]:
test_acc, _ = eval_model(
  model,
  test_data_loader,
  loss_fn,
  device,
  len(test) #Change it to test when you have the test results
)
test_acc.item()

In [ ]:
def get_predictions(model, data_loader):
  model = model.eval()
  sentence = []
  predictions = []
  prediction_probs = []
  real_values = []
  with torch.no_grad():
    for d in data_loader:
      texts = d["text"]
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      labels = d["label"].to(device)
      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      _, preds = torch.max(outputs, dim=1)
      sentence.extend(texts)
      predictions.extend(preds)
      prediction_probs.extend(outputs)
      real_values.extend(labels)
  predictions = torch.stack(predictions).cpu()
  prediction_probs = torch.stack(prediction_probs).cpu()
  real_values = torch.stack(real_values).cpu()
  return sentence, predictions, prediction_probs, real_values

In [ ]:
y_review_texts, y_pred, y_pred_probs, y_test = get_predictions(
  model,
  test_data_loader
)

In [ ]:
class_name = ['CL' ,'CR' ,'DC' ,'DS' ,'LO' , 'NI' , 'SE' ]

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test, y_pred, target_names=class_name,zero_division=0))

In [ ]:
import seaborn as sns
def show_confusion_matrix(confusion_matrix):
  hmap = sns.heatmap(confusion_matrix, annot=True, fmt="d", cmap="Reds")
  hmap.yaxis.set_ticklabels(hmap.yaxis.get_ticklabels(), rotation=0, ha='right')
  hmap.xaxis.set_ticklabels(hmap.xaxis.get_ticklabels(), rotation=30, ha='right')
  plt.ylabel('True sentiment')
  plt.xlabel('Predicted sentiment')
cm = confusion_matrix(y_test, y_pred)
df_cm = pd.DataFrame(cm, index=class_name, columns=class_name)
show_confusion_matrix(df_cm)

In [ ]:
scibert_preds = pd.DataFrame(list(zip(y_review_texts,y_test.tolist(),y_pred.tolist())),
                          columns = ['text','true_labels','pred_labels'])

scibert_preds.to_csv('scibert_preds.csv')

In [ ]:
scibert_preds.tail(4)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# reduce to 1d array
#yhat_probs = yhat_probs[:,0]
#yhat_classes = yhat_classes[:,0]
#accuracy = accuracy_score(y_test, yhat_classes)
#print('Accuracy: %f' % accuracy)
# accuracy: (tp + tn) / (p + n)
# precision tp / (tp + fp)
precision = precision_score(y_test, y_pred,average='micro')
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, y_pred,average='micro')
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test, y_pred,average='micro')
print('F1 score: %f' % f1)